In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd gdrive/MyDrive/Remi/

Mounted at /content/gdrive
/content/gdrive/.shortcut-targets-by-id/1WJLnwO7mUn1QFbhFIJ6fMZjZWuCKf1ub/Remi


In [2]:
!pip install transformers
!pip install sentencepiece
!pip install sentence_transformers
!pip install fairseq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import torch
import pandas as pd
import re
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

#model_name = "roberta-large-mnli"
#model_name = "tals/albert-xlarge-vitaminc-mnli"
#model_name = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
#model_name = "MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli"

model_name = "vicgalle/xlm-roberta-large-xnli-anli"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

#premise = "Read more about how the EU, UK, US and others are blocking the expansion of vaccine production at the WTO, thus prolonging the pandemic. From the WTO: https://t.co/tHLPGjC8DY Detailed Q&amp;A from Human Rights Watch: https://t.co/3vOYZT2Fae https://t.co/zNZzrT6DF6"
#hypothesis = "Intellectual property The first polio vaccine was never patented; some have argued that similar treatment of an effective COVID‑19 vaccine could enable fair distribution.Initially, negotiations at the World Trade Organization (WTO) on the issue of waiving patent rights were blocked for months by resistance by the US, Switzerland, Norway, and the EU. Initially one observer considered the US position unlikely to change, but as of April 2021 the US administration was discussing the issue and then reversed course and announced its support for a patent waiver for COVID-19 vaccines on 5 May 2021. 400 non-profit organizations and 115 members of the European Commission have signed a letter urging the United States and Europe to side with the WTO members in the global south."
# premise = "If these so-called vaccines cannot guarantee no transmission of Covid-19 there is absolutely no justification for taking the risk. The people who have suppressed and smeared hydroxychloroquine and ivermectin have caused Americans to die. They need to be arrested and these"
# hypothesis = "Hydroxychloroquine, sold under the brand name Plaquenil among others, is a medication used to prevent and treat malaria in areas where malaria remains sensitive to chloroquine. Other uses include treatment of rheumatoid arthritis, lupus, and porphyria cutanea tarda. It is taken by mouth, often in the form of hydroxychloroquine sulfate.Common side effects may include vomiting, headache, changes in vision, and muscle weakness. Severe side effects may include allergic reactions, vision problems, and heart problems. Although all risk cannot be excluded, it remains a treatment for rheumatic disease during pregnancy. Hydroxychloroquine is in the antimalarial and 4-aminoquinoline families of medication.Hydroxychloroquine was approved for medical use in the United States in 1955. It is on the World Health Organization's List of Essential Medicines. In 2019, it was the 122nd most commonly prescribed medication in the United States, with more than 5 million prescriptions.Hydroxychloroquine has been studied for an ability to prevent and treat coronavirus disease 2019 (COVID-19), but clinical trials found it ineffective for this purpose and a possible risk of dangerous side effects. Among studies that deemed hydroxychloroquine intake to cause harmful side effects, a publication by The Lancet was retracted due to data flaws. The speculative use of hydroxychloroquine for COVID-19 threatens its availability for people with established indications."
# premise = 'I like you'
# hypothesis = 'I hate you'

with torch.inference_mode():
  out = model(**tokenizer('I like you', 'I hate you', return_tensors='pt').to(model.device))
  proba = torch.softmax(out.logits, -1).cpu().numpy()[0]
  print({v: proba[k] for k, v in model.config.id2label.items()})
  print(proba.argsort()[-1])
  print('-----------------------------------------')
  out = model(**tokenizer('When did he leave?', 'he left', return_tensors='pt').to(model.device))
  proba = torch.softmax(out.logits, -1).cpu().numpy()[0]
  print({v: proba[k] for k, v in model.config.id2label.items()})
  print(proba.argsort()[-1])
  print('-----------------------------------------')

index = 0
global match
match = 0

def predict_nli(prediction, nli_label):
  global match
  if prediction == 0:
    print("refute", nli_label)
    if nli_label == "refute":
      match += 1
  elif prediction == 1:
    print("neutral", nli_label)
    if nli_label == "neutral":
      match += 1
  elif prediction == 2:
    print("support", nli_label)
    if nli_label == "support":
      match += 1
  else:
    print("----------------------")

for checkworthy in covid_check['class_label']:
  if checkworthy == 1:
    if type(covid_check['verify_url'][index]) == str:
      verify_text = covid_check['verify_text'][index].split('\n\n')
      verify_text = [re.sub('\s+', ' ', _) for _ in verify_text]
      verify_text = [re.sub('  ', '', _) for _ in verify_text]
      if len(verify_text) == 1:
        with torch.inference_mode():
          verify_text = covid_check['verify_text'][index].split()
          verify_text = verify_text[:128]
          print(len(verify_text))
          verify_text = " ".join(str(x) for x in verify_text)
          out = model(**tokenizer(covid_check['tweet_text'][index], verify_text, return_tensors='pt').to(model.device))
          proba = torch.softmax(out.logits, -1).cpu().numpy()[0]
          prediction = proba.argsort()[-1]
          predict_nli(prediction, covid_check['NLI'][index])
      elif len(verify_text) > 1:
        nli_labels = covid_check['NLI'][index].split('\n\n')
        i = 0
        for text in verify_text:
          with torch.inference_mode():
            text = text.split()
            text = text[:128]
            print(len(text))
            text = " ".join(str(x) for x in text)
            out = model(**tokenizer(covid_check['tweet_text'][index], text, return_tensors='pt').to(model.device))
            proba = torch.softmax(out.logits, -1).cpu().numpy()[0]
            prediction = proba.argsort()[-1]
            predict_nli(prediction, nli_labels[i])
          i+= 1
  index += 1

print("Amount correct: ", match)


{'contradiction': 0.9994704, 'neutral': 0.00031146815, 'entailment': 0.00021811709}
0
-----------------------------------------
{'contradiction': 0.00021974502, 'neutral': 0.00027642882, 'entailment': 0.99950385}
2
-----------------------------------------
128
neutral refute
127
support support
67
neutral support
86
support refute
22
neutral support
128
neutral refute
128
neutral refute
128
neutral neutral
128
support neutral
128
neutral neutral
64
neutral neutral
128
neutral neutral
128
support refute
128
neutral refute
128
support neutral
96
support support
128
neutral refute
128
support refute
33
neutral neutral
67
neutral support
128
support neutral
128
refute refute
128
refute refute
128
support neutral
128
neutral neutral
128
support support
128
support support
128
support refute
27
support refute
70
neutral support
128
neutral support
128
support support
128
neutral refute
128
support neutral
128
support refute
128
neutral refute
128
support support
77
support neutral
128
neutra

In [10]:
import pandas as pd

#!pip install fairseq
covid_check = pd.read_csv('no_headings_nl_merged_clef_emnlp_downsampled_to_en - Sheet1.csv', sep=',')
covid_check['verify_text']
covid_check['tweet_text']
covid_check['NLI?']

roberta = torch.hub.load('pytorch/fairseq', 'bart.large.mnli')
roberta.eval()

index = 0
global match
match = 0


def predict_nli(prediction, nli_label):
  global match
  if prediction == 0:
    print("Refute", nli_label)
    if nli_label == "Refute":
      match += 1
  elif prediction == 1:
    print("neutral", nli_label)
    if nli_label == "neutral":
      match += 1
  elif prediction == 2:
    print("support", nli_label)
    if nli_label == "support":
      match += 1
  else:
    print("----------------------")

for checkworthy in covid_check['class_label']:
  if checkworthy == 1:
    if type(covid_check['verify_url'][index]) == str:
      verify_text = covid_check['verify_text'][index].split('\n\n')
      verify_text = [re.sub('\s+', ' ', _) for _ in verify_text]
      verify_text = [re.sub('  ', '', _) for _ in verify_text] 
      if len(verify_text) == 1:
        tokens = roberta.encode(covid_check['tweet_text'][index], covid_check['verify_text'][index])
        tokens = tokens[:512]
        prediction = roberta.predict('mnli', tokens).argmax()
        predict_nli(prediction, covid_check['NLI?'][index])
      elif len(verify_text) > 1:
        nli_labels = covid_check['NLI?'][index].split('\n\n')
        i = 0
        for text in verify_text:
          tokens = roberta.encode(covid_check['tweet_text'][index], text)
          tokens = tokens[:512]
          prediction = roberta.predict('mnli', tokens).argmax()
          predict_nli(prediction, nli_labels[i])
          i+= 1
  index += 1

print("Amount correct: ", match)


KeyError: ignored

In [40]:

import os
import pandas
from torch.utils.data import DataLoader
import math
from sentence_transformers import LoggingHandler, util
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CESoftmaxAccuracyEvaluator
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import numpy as np
import csv

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout


# As dataset, we use SNLI + MultiNLI
# Check if dataset exsist. If not, download and extract  it

dataset_path = 'data/train.csv'

# Read the AllNLI.tsv.gz file and create the training dataset
logging.info("Read AllNLI train dataset")

label2int = {"contradiction": 0, "entailment": 1, "neutral": 2}
train_samples = []
dev_samples = []
train_data = pandas.read_csv(dataset_path)
train_data['label'] = train_data['label'].replace([0, 2], [2, 0])
for id, row in train_data.iterrows():
    label_id = int(row['label'])
    train_samples.append(InputExample(texts=[row['premise'], row['hypothesis']], label=label_id))

train_batch_size = 16
num_epochs = 3
model_save_path = './training_allnli-' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Define our CrossEncoder model. We use distilroberta-base as basis and setup it up to predict 3 labels
model = CrossEncoder('sentence-transformers/distilbert-multilingual-nli-stsb-quora-ranking', num_labels=len(label2int))
#model = CrossEncoder('sentence-transformers/xlm-r-100langs-bert-base-nli-mean-tokens', num_labels=len(label2int))
# model = CrossEncoder('xlm-roberta-base', num_labels=len(label2int))

# Trye smaller batch size
# model = CrossEncoder('joeddav/xlm-roberta-large-xnli', num_labels=len(label2int))

# We wrap train_samples, which is a list ot InputExample, in a pytorch DataLoader
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)

# During training, we use CESoftmaxAccuracyEvaluator to measure the accuracy on the dev set.
#evaluator = CESoftmaxAccuracyEvaluator.from_input_examples(dev_samples, name='AllNLI-dev')

warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)  # 10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

# Train the model
model.fit(train_dataloader=train_dataloader,
          epochs=num_epochs,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

test_dataset = 'data/test - test.csv'
df = pandas.read_csv(test_dataset)
sentence_pairs = []
ids = []
hypothesis = []
premise = []
for id, row in df.iterrows():
    label_id = 0
    ids.append(row['id'])
    hypothesis.append(row['hypothesis'])
    premise.append(row['premise'])
    sentence_pairs.append([row['premise'], row['hypothesis']])

pred_scores = model.predict(sentence_pairs, convert_to_numpy=True, show_progress_bar=False, 
                            batch_size=train_batch_size)
pred_labels = np.argmax(pred_scores, axis=1)

out_df = pandas.DataFrame([ids, pred_labels, hypothesis, premise]).transpose()
out_df = out_df.rename(columns={0: 'id', 1: 'prediction', 2: 'hypothesis', 3: 'premise'})
out_df['prediction'] = out_df['prediction'].replace([0, 2],[2, 0])

out_df.to_csv('submission.csv', index=False)

# --------------------------------------------------------------------------------------------------------


pred_scores = model.predict(['Why did he leave', 'he left'], convert_to_numpy=True, show_progress_bar=False, 
                            batch_size=train_batch_size)

print(np.argmax(pred_scores))
print('---------------------------------------------------------------------------------------')

pred_scores = model.predict(['I love you', 'I hate you'], convert_to_numpy=True, show_progress_bar=False, 
                            batch_size=train_batch_size)

print(np.argmax(pred_scores))
print('---------------------------------------------------------------------------------------')

index = 0
global match
match = 0

def predict_nli(prediction, nli_label):
  global match
  if prediction == 0:
    print("support", nli_label)
    if nli_label == "support":
      match += 1
  elif prediction == 1:
    print("neutral", nli_label)
    if nli_label == "neutral":
      match += 1
  elif prediction == 2:
    print("refute", nli_label)
    if nli_label == "refute":
      match += 1
  else:
    print("----------------------")

for checkworthy in covid_check['class_label']:
  if checkworthy == 1:
    if type(covid_check['verify_url'][index]) == str:
      verify_text = covid_check['verify_text'][index].split('\n\n')
      verify_text = [re.sub('\s+', ' ', _) for _ in verify_text]
      verify_text = [re.sub('  ', '', _) for _ in verify_text]
      if len(verify_text) == 1:
          sentence_pair = [verify_text, covid_check['tweet_text'][index]]
          pred_scores = model.predict(sentence_pair, convert_to_numpy=True, show_progress_bar=False, batch_size=1)
          pred_labels = np.argmax(pred_scores, axis=1)
          predict_nli(pred_labels, covid_check['NLI?'][index])
      elif len(verify_text) > 1:
        nli_labels = covid_check['NLI?'][index].split('\n\n')
        i = 0
        for text in verify_text:
          sentence_pair = [text, covid_check['tweet_text'][index]]
          pred_scores = model.predict(sentence_pair, convert_to_numpy=True, show_progress_bar=False, batch_size=1)
          pred_labels = np.argmax(pred_scores, axis=1)
          predict_nli(pred_labels, nli_labels[i])
          i+= 1
  index += 1

print("Amount correct: ", match)





Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/distilbert-multilingual-nli-stsb-quora-ranking and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/758 [00:00<?, ?it/s]

Iteration:   0%|          | 0/758 [00:00<?, ?it/s]

Iteration:   0%|          | 0/758 [00:00<?, ?it/s]

0
---------------------------------------------------------------------------------------
0
---------------------------------------------------------------------------------------


TypeError: ignored